<a href="https://colab.research.google.com/github/AprilisMaijs/bakalaurs-twitter-sentiment/blob/neuralnet_training/LVTwitSentimenti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing and importing dependencies

In [ ]:
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu117


In [ ]:
!pip3 install transformers snscrape beautifulsoup4 pandas numpy

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

In [ ]:
pip install datasets

In [ ]:
pip install evaluate

# Training data preprocessing


In [ ]:
df_train = pd.read_csv('/content/drive/MyDrive/bakalaurs/marked_train.txt', delimiter='|',encoding = 'UTF-8')
df_eval = pd.read_csv('/content/drive/MyDrive/bakalaurs/marked_eval.txt', delimiter='|',encoding = 'UTF-8')
df_train.columns = ['LABEL_COLUMN', 'DATA_COLUMN']
df_eval.columns = ['LABEL_COLUMN', 'DATA_COLUMN']

In [ ]:
display(df_train)
display(df_eval)

In [ ]:
df_train = df_train.sort_values(by=['LABEL_COLUMN'])

In [ ]:
df_train.sample(10)

In [ ]:
#regexes - Alīna Govoruhina(2022)
newLine ="\\n|\\r"
urls = '(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,})'
numbers = '\d+((\.|\-)\d+)?'
mentions = '\B\@([\w\-]+)'
hashtag = '#'
whitespaces = '\s+'
leadTrailWhitespace = '^\s+|\s+?$'


#combining regexes into single 
combined_pattern = f'{newLine}|{urls}|{numbers}|{mentions}|{hashtag}'
df_train['DATA_COLUMN'] = df_train['DATA_COLUMN'].str.replace(combined_pattern, '')
df_eval['DATA_COLUMN'] = df_eval['DATA_COLUMN'].str.replace(combined_pattern, '')


In [ ]:
df_train = df_train[['DATA_COLUMN', 'LABEL_COLUMN']]
df_eval = df_eval[['DATA_COLUMN', 'LABEL_COLUMN']]

In [ ]:
df_train.sample(10)

# Hugging Face fine-tuning guide


In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

model = AutoModelForSequenceClassification.from_pretrained("AiLab-IMCS-UL/lvbert", num_labels=3)
tokenizer = BertTokenizer.from_pretrained("AiLab-IMCS-UL/lvbert")

In [ ]:
import tensorflow as tf
from datasets import Dataset
import pandas as pd
ds_train = Dataset.from_pandas(df_train)
ds_test = Dataset.from_pandas(df_eval)

In [ ]:
ds_train["DATA_COLUMN"][:10]

In [ ]:
from transformers import AutoTokenizer

tokenizer = BertTokenizer.from_pretrained("AiLab-IMCS-UL/lvbert")


def tokenize_function(examples):
    return tokenizer(examples["DATA_COLUMN"], padding="max_length", truncation=True)


tokenized_datasets = ds_train.map(tokenize_function, batched=True)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_train,
    eval_dataset=ds_test,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

# Instantiate LVBERT model 'AiLab-IMCS-UL/lvbert'



In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

model = TFBertForSequenceClassification.from_pretrained("AiLab-IMCS-UL/lvbert", from_pt=True)
tokenizer = BertTokenizer.from_pretrained("AiLab-IMCS-UL/lvbert")

In [ ]:
model.summary()

1 — convert_data_to_examples: This will accept our train and test datasets and convert each row into an InputExample object.

2 — convert_examples_to_tf_dataset: This function will tokenize the InputExample objects, then create the required input format with the tokenized objects, finally, create an input dataset that we can feed to the model.


In [ ]:
import tensorflow as tf

def convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN): 
  train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

  validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)
  
  return train_InputExamples, validation_InputExamples

  train_InputExamples, validation_InputExamples = convert_data_to_examples(train, 
                                                                           test, 
                                                                           'DATA_COLUMN', 
                                                                           'LABEL_COLUMN')
  
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )


DATA_COLUMN = 'DATA_COLUMN'
LABEL_COLUMN = 'LABEL_COLUMN'

In [ ]:
train = df_train
test = df_eval

In [ ]:
train.sample(10)

In [ ]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

In [ ]:
pred_sentences = ['Iespējams sliktākā partija pasaulē - nekādas atbidības, naudas izšķērdēšana un korupcija valda visu!','Paldies partijai par jaunajām ietvēm! Manam kaķim un man arī ļoti patīk!']

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_data, epochs=2, validation_data=validation_data)

# Instantiate mBERT model


In [ ]:
tokens = tokenizer.encode('Man riebās. Pretīga izrāde un nepatīkami aktieri.', return_tensors='pt')

In [ ]:
result = model(tokens)

In [ ]:
result.logits

In [ ]:
int(torch.argmax(result.logits))

# Collecting tweets WITHOUT API



In [ ]:
import os
import pandas as pd
import snscrape.modules.twitter as sntwitter
from datetime import date

In [ ]:

tweet_data = []

#query = input('enter your keyword: ')
query = 'vienotība since:2022-11-28 until:2022-12-31'
number = int(input('how many tweets to scrape: '))
for i, tweets in enumerate(sntwitter.TwitterSearchScraper('{}'.format(query)).get_items()):
    if i > number:
        break
    tweet_data.append([tweets.date, tweets.content, tweets.username, tweets.url])

df = pd.DataFrame(tweet_data, columns = ['Date', 'Tweets', 'Username', 'URL'])
df.to_csv(f'{query}.csv', index=False, encoding='utf-8')

